# Team Flat Ocean Saviors
## Software Architecture - Taking and Uploading the Picture 

---

The purpose of this file is to walk through our current code in enabling the Pi to capture and upload a picture to GitHub. An outline of each step is attached below: 

- [1. Imports](#1)
- [2. Setup](#2)
- [3. Taking the Picture](#3)
- [4. Uploading to GitHub](#4)

---

## <a id =1> </a> 1. Imports

Below are the required imports for our code. We decided to not use the ```git``` module in favor of ```os```. Due to ```picamera```, this code can only be executed on a RaspberryPi. 

In [ ]:
import time
import os
import board
import busio
import adafruit_bno055
#from git import Repo
from picamera import PiCamera

## <a id =2> </a> 2. Setup

Variables ```i2c``` and ```sensor``` initiate the acceleration sensor, and have come with the template. ```threshold``` is in preparation for the minimum acceleration needed for the Pi to take a picture, for which we decided to use 10 meters per second squared. ```newCamera``` creates a new object of class PiCamera, whose associated methods will be used to take the picture in the next section.

In [ ]:
i2c = busio.I2C(board.SCL, board.SDA)
sensor = adafruit_bno055.BNO055_I2C(i2c)

#SET THRESHOLD
threshold = 10

newCamera = PiCamera()

## <a id =3> </a> 3. Taking the Picture

```accelX, accelY, accelZ = sensor.acceleration``` is how we will access the X, Y, and Z accelerations detected by the sensor.<br>
<br>
```if abs(accelX) > threshold or abs(accelY) > threshold or abs(accelZ-9.81) > threshold:``` states that if X, Y, or Z detects an acceleration of greater than the threshold (10), then the picture will be taken. For Z, 9.81 must be subtracted from the read value in order for the code to not be mistakenly triggered by gravitational acceleration.<br>
<br>
```name = "SaviorsO"     #Last Name, First Initial  ex. FoxJ``` is required by the template, for which ```name``` will be used in the filename of the picture.<br>
```python 
if name:
    newCamera.start_preview()
    print("Preparing: Please Wait")
    time.sleep(5)
``` 
The above initializes the camera. The camera requires at least 3 seconds in order for the lenses to calibrate. To be completely sure that it is ready, we decided to give the camera 5 seconds instead (```time.sleep(5)```). <br>
```python3 
t = time.strftime("_%H%M%S")      # current time string
imgname = f"{name}{t}" #change directory to your folder
```
The above is purely for the picture filename. The picture has not been taken yet, but when it is, the filename will already be ready due to this code. ```t``` is holds the current time. ```imgname``` combines ```name``` and ```t``` into a filename presenting the information stored in both variables. The file will be located in the current working directory. <br>
<br>
```newCamera.capture(imgname) #saves image into a file``` is a method of class PiCamera that turns what the camera is currently seeing into a file in the current working directory of name ```imgname```, the name that was compiled in the previous part of the code. <br>
<br>
```git_push()``` is the final step, where the image will be uploaded to GitHub. This will be explored in the next section. 

In [ ]:
#read acceleration
while True:
    accelX, accelY, accelZ = sensor.acceleration

    #CHECK IF READINGS ARE ABOVE THRESHOLD
    if abs(accelX) > threshold or abs(accelY) > threshold or abs(accelZ-9.81) > threshold:
        #PAUSE
        time.sleep(0.1)

        #TAKE/SAVE/UPLOAD A PICTURE 
        name = "SaviorsO"     #Last Name, First Initial  ex. FoxJ
        
        if name:
            newCamera.start_preview()
            print("Preparing: Please Wait")
            time.sleep(5)
            t = time.strftime("_%H%M%S")      # current time string
            imgname = ('/home/pi/Desktop/newImage%s%s.jpg' % (name,t)) #change directory to your folder
    
            #<YOUR CODE GOES HERE>#
            newCamera.capture(imgname) #saves image into a file
            print("Complete. Now Uploading")
            git_push()
            print("Completed Upload")
    
    #PAUSE
    time.sleep(1)

## <a id =4> </a> 4. Uploading to GitHub

The following code executes commands in Terminal to upload this folder to GitHub, with the taken picture. ```os.system(arg)``` runs the argument exactly as if you were running the command yourself in the Terminal.

In [ ]:
def git_push():
    try:
        os.system("git add -A")
        msg = input("add message: ")
        os.system(f"git commit -m {msg}")
        os.system("git push")
    except:
        print("Couldn't upload to git")